## Almacenes de datos

Concepto
Los almacenes de datos de AzureML son abstracciones para fuentes de datos en la nube.

Funciones:
- Encapsulan información para la conexión a fuentes de datos.
- Permiten acceso directo a través del SDK de AzureML.
- Facilitan la carga y descarga de datos.

Tipos de Almacenes de Datos Soportados:
- Azure Blob Storage
- Azure File Storage
- Azure Data Lake Store
- Azure SQL Database
- Azure Databricks File System (DBFS)
- Entre los mas usados.. (Para la lista completa, consulte la documentación de AzureML)

Almacenes de Datos Predeterminados:
- Cada espacio de trabajo tiene dos almacenes predeterminados:
  - Contenedor Blob de Azure Storage
  - Contenedor de archivos de Azure Storage
- Se utilizan para el almacenamiento del sistema de AzureML.

Tercer Almacén de Datos:
Se agrega automáticamente al usar conjuntos de datos de ejemplo.

Uso en Proyectos:
- En la mayoría de los proyectos, necesitará usar sus propias fuentes de datos.Las razones pueden ser:
  1. Manejar mayores volúmenes de datos.
  2. Integrar la solución con datos de aplicaciones existentes.

## Crear y administrar Almacenes de datos

#### Registro:

- Interfaz gráfica de AzureML Studio.
- SDK de AzureML.

In [ ]:
# Registrar un contenedor Blob de Azure Storage llamado "blob_data"
from azureml.core import Workspace, Datastore

ws = Workspace.from_config()

# Registramos el nuevo Datastore
blob_ds = Datastore.register_azure_blob_container(
    workspace=ws, 
    datastore_name='blob_data', 
    container_name='data', 
    account_name='storageaccountname', 
    container_name='data_container', 
    account_name='az_store...', 
    account_key='storageaccountkey'
    )

 #### Visualización y Gestión:

Realizable mediante:
- AzureML Studio.
- SDK de AzureML.

Ejemplo: Listar nombres de almacenes con un bucle for y obtener referencias individuales con el método get

In [ ]:
# Listamos los datastores registrados
for ds_name in ws.datastores:
    print(ds_name)

: 

#### Almacenamiento Predeterminado:

Incluido en todo espacio de trabajo que se crea.
- Recuperable con el método get_default_datastore.
- Inicialmente es "workspace blob store datastore".

In [ ]:
# Obtenemos el datastore recién registrado
blob_store = Datastore.get(ws, datastore_name='blob_data')

#### Consideraciones para la Planificación:

Azure Blob Storage:
- Almacenamiento premium: mejor rendimiento I/O para grandes conjuntos de datos (mayor costo y limitaciones de replicación).

Formato de Archivos:
- Parquet generalmente ofrece mejor rendimiento que CSV.

Acceso y Cambio de Predeterminado:
- Acceso por nombre del almacén de datos.
- Cambio de predeterminado con el método set_default_datastore.

In [ ]:
# configuramos el datastore por defecto
ws.set_default_datastore('blob_data')

### Ejercicio practico - trabajar con datasets

##### Creacion, carga y versiones
1. Creación de conjuntos de datos de archivos:
   
    Los datasets son objetos de datos empaquetados con versiones que se pueden usar en experimentos y pipelines. Son la forma recomendada para trabajar con datos en Azure Machine Learning.

    Tipos:

    - **Tabular**: Para datos con estructura consistente (ej. DataFrames de Pandas).
    - **Archivo**: Para datos no estructurados o procesamiento a nivel de archivo (ej. imágenes).

    El código en Python muestra cómo crear un dataset tabular a partir de dos rutas de archivo y registrarlo en el espacio de trabajo.1

In [ ]:
# Para crear un conjunto de datos tabular mediante el SDK, se usa el  método from_delimited_files de la  clase Dataset.Tabular, como se muestra a continuación:

from azureml.core import Dataset

blob_ds = ws.get_default_datastore()

csv_paths = [(blob_ds, 'data/files/current_data.csv'),          # Archivo current_data.csv en la  carpeta data/files/   
            (blob_ds, 'data/files/archive/*.csv')]              # Todos los archivos .csv en la  carpeta data/files/archive/

# Usando el método from_delimited_files
tab_ds = Dataset.Tabular.from_delimited_files(path=csv_paths)   # from_delimited_files de la  clase Dataset.Tabular

# 
tab_ds = tab_ds.register(workspace=ws, name='csv_table')

: 

In [ ]:
# Para crear un conjunto de datos de archivo mediante el SDK, use el  método from_files de la  clase Dataset.File

from azureml.core import Dataset

blob_ds = ws.get_default_datastore()

file_ds = Dataset.File.from_files(path=(blob_ds, 'data/files/images/*.jpg'))    # from_files de la  clase Dataset.File

file_ds = file_ds.register(workspace=ws, name='img_files')

: 

2. Recuperación de conjuntos de datos registrados:

    Se puede recuperar el dataset mediante cualquiera de las siguientes técnicas:

    - El  atributo `datasets` dictionary de un objeto Workspace.
    - Método `get_by_name` o `get_by_id` de la clase Dataset.

In [ ]:
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()                        # Obtenemos el objeto Workspace

ds1 = ws.datasets['csv_table']                      # Seleccionamos el conjunto de datos por su nombre 

ds2 = Dataset.get_by_name(ws, 'img_files')          # Seleccionamos el conjunto de datos por el nombre la su clase

3. Versionado de conjuntos de datos:

    Los conjuntos de datos se pueden versionar, lo que le permite realizar un seguimiento de las versiones históricas y reproducir esos experimentos con datos en el mismo estado.

    Puede crear una nueva versión de un conjunto de datos registrándolo con el mismo nombre que un conjunto de datos registrado anteriormente y especificando la  propiedad `create_new_version`

In [ ]:
# En este ejemplo, los archivos .png de la carpeta images se han agregado al conjunto de datos img_paths utilizado.

img_paths = [(blob_ds, 'data/files/images/*.jpg'),
             (blob_ds, 'data/files/images/*.png')]

file_ds = Dataset.File.from_files(path=img_paths)                                   # Usamos el método from_files para registrar los archivos jpg y png

file_ds = file_ds.register(workspace=ws, name='img_files', create_new_version=True) # creamos una nueva versión del conjunto de datos img_files

4. Recuperación de versiones específicas de conjuntos de datos:

    Podemos recuperar una versión específica de un conjunto especificando el parámetro version en el método `get_by_name` de la clase Dataset.
    

In [ ]:
img_ds = Dataset.get_by_name(workspace=ws, name='img_files', version=2)            # Obtenemos la versión 2 del conjunto de datos img_files

##### Manejo y uso

5. Trabajar con conjuntos de datos 

    Podemos leer datos directamente de un conjunto de datos tabular convirtiéndolos en un marco de datos de Pandas o Spark

In [ ]:

df = tab_ds.to_pandas_dataframe()
print(df.head()

6. Pasar un conjunto de datos tabular a un script
   
Cuando necesitemos acceder a un conjunto de datos mediante un script, debemos pasarle el conjunto de datos al script. 

Hay dos maneras de hacerlo:

- **Usar un argumento de script para un conjunto de datos tabulares**

    Podemos pasar el conjunto de datos como argumento de script. 
    
    Cuando se adopta este enfoque, el argumento recibido por el script es el identificador único del conjunto de datos en el área de trabajo. En el script, puede obtener el área de trabajo del contexto de ejecución y usarlo para recuperar el conjunto de datos por su identificador.

In [ ]:
# ScriptRunConfig

env = Environment('my_env')                                                         # Creamos un objeto Environment  
packages = CondaDependencies.create(conda_packages=['pip'],                         # Creamos un objeto CondaDependencies con las dependencias necesarias
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages                                            # Asignamos las dependencias al entorno                             

script_config = ScriptRunConfig(source_directory='my_dir',                          # Creamos un objeto ScriptRunConfig   
                                script='script.py',
                                arguments=['--ds', tab_ds],)

In [ ]:
# Script

from azureml.core import Run, Dataset

parser.add_argument('--ds', type=str, dest='dataset_id')                            # Agregamos un argumento para el conjunto de datos
args = parser.parse_args()

run = Run.get_context()
ws = run.experiment.workspace
dataset = Dataset.get_by_id(ws, id=args.dataset_id)                                 # Obtenemos el conjunto de datos por su id
data = dataset.to_pandas_dataframe()

- **Usar una entrada con nombre para un conjunto de datos tabular**

    Como alternativa, podemos pasar un conjunto de datos tabular como una entrada con nombre. En este enfoque, se utiliza el  método `as_named_input` del conjunto de datos para especificar un nombre. 

    A continuación, en el script, puede recuperar el conjunto de datos por nombre de la colección `input_datasets` del contexto de ejecución sin necesidad de recuperarlo del área de trabajo. Tenga en cuenta que si usa este enfoque, aún debe incluir un argumento de script para el conjunto de datos, aunque en realidad no lo use para recuperar el conjunto de datos.

In [ ]:
#ScriptRunConfig

env = Environment('my_env')
packages = CondaDependencies.create(conda_packages=['pip'],
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages

script_config = ScriptRunConfig(source_directory='my_dir',
                                script='script.py',
                                arguments=['--ds', tab_ds.as_named_input('my_dataset')],    # Pasamos el conjunto de datos como argumento
                                environment=env)

In [ ]:
# Script

from azureml.core import Run

parser.add_argument('--ds', type=str, dest='ds_id')
args = parser.parse_args()

run = Run.get_context()
dataset = run.input_datasets['my_dataset']                                          # Obtenemos el conjunto de datos por su nombre
data = dataset.to_pandas_dataframe()

7. Trabajar con conjuntos de datos de archivos

    Para trabajar con un conjunto de datos de archivos, podemos utilizar el método `to_path()` para devolver una lista de las rutas de archivo encapsuladas por el conjunto de datos

In [ ]:
for file_path in file_ds.to_path():
    print(file_path)

8. Pasar un conjunto de datos de archivo a un script 

    Al igual que con los conjuntos de datos tabulares, hay dos formas de pasar un conjunto de datos de archivo a un script. Sin embargo, hay algunas diferencias clave en la forma en que se pasa el conjunto de datos.

    - **Usar un argumento de script para un conjunto de datos de archivos**

        Puede pasar un conjunto de datos de archivos como argumento de script. A diferencia de lo que ocurre con un conjunto de datos tabular, debe especificar un modo para el argumento del conjunto de datos de archivo, que puede ser `as_download` o `as_mount`. Esto proporciona un punto de acceso que el script puede utilizar para leer los archivos del conjunto de datos. 

        En la mayoría de los casos, se ha de `as_download`, que copia los archivos en una ubicación temporal en el proceso donde se ejecuta el script. Sin embargo, si está trabajando con una gran cantidad de datos para los que es posible que no haya suficiente espacio de almacenamiento en el proceso, usaremos `as_mount` para transmitir los archivos directamente desde su origen.

In [ ]:
#ScriptRunConfig

env = Environment('my_env')
packages = CondaDependencies.create(conda_packages=['pip'],
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages

script_config = ScriptRunConfig(source_directory='my_dir',
                                script='script.py',
                                arguments=['--ds', file_ds.as_download()],                          # Pasamos el conjunto de datos como argumento usando el método as_download
                                environment=env)

In [ ]:
#Script

from azureml.core import Run
import glob

parser.add_argument('--ds', type=str, dest='ds_ref')
args = parser.parse_args()
run = Run.get_context()

imgs = glob.glob(args.ds_ref + "/*.jpg")                                                            # Obtenemos los archivos del conjunto de datos

- **Usar una entrada con nombre para un conjunto de datos de archivos**

    También puede pasar un conjunto de datos de archivos como una entrada con nombre. En este enfoque, se utiliza el  método as_named_input del conjunto de datos para especificar un nombre antes de especificar el modo de acceso. A continuación, en el script, puede recuperar el conjunto de datos por nombre de la colección input_datasets del contexto de ejecución  y leer los archivos desde allí. 

    Al igual que con los conjuntos de datos tabulares, si usa una entrada con nombre, debe incluir un argumento de script para el conjunto de datos, aunque en realidad no lo use para recuperar el conjunto de datos.

In [ ]:
# ScriptRunConfig

env = Environment('my_env')
packages = CondaDependencies.create(conda_packages=['pip'],
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages

script_config = ScriptRunConfig(source_directory='my_dir',
                                script='script.py',
                                arguments=['--ds', file_ds.as_download()],                       # Pasamos el conjunto de datos como argumento usando el método as_download
                                environment=env)

In [ ]:
# Script

from azureml.core import Run
import glob

parser.add_argument('--ds', type=str, dest='ds_ref')
args = parser.parse_args()
run = Run.get_context()

imgs = glob.glob(args.ds_ref + "/*.jpg")                                                           # Obtenemos los archivos del conjunto de datos

11. Usar una entrada con nombre para un conjunto de datos de archivos
    
    En este enfoque, se utiliza el  método `as_named_input` del conjunto de datos para especificar un nombre antes de especificar el modo de acceso. A continuación, en el script, puede recuperar el conjunto de datos por nombre de la colección `input_datasets` del contexto de ejecución y leer los archivos desde allí. 

    Al igual que con los conjuntos de datos tabulares, si usa una entrada con nombre, debe incluir un argumento de script para el conjunto de datos, aunque en realidad no lo use para recuperar el conjunto de datos.

In [ ]:
# ScriptRunConfig

env = Environment('my_env')
packages = CondaDependencies.create(conda_packages=['pip'],
                                    pip_packages=['azureml-defaults',
                                                  'azureml-dataprep[pandas]'])
env.python.conda_dependencies = packages

script_config = ScriptRunConfig(source_directory='my_dir',
                                script='script.py',
                                arguments=['--ds', file_ds.as_named_input('my_ds').as_download()],    # Pasamos el conjunto de datos como argumento usando el método as_named_input
                                environment=env)

In [ ]:
# Script

from azureml.core import Run
import glob

parser.add_argument('--ds', type=str, dest='ds_ref')
args = parser.parse_args()
run = Run.get_context()

dataset = run.input_datasets['my_ds']                                                        # Obtenemos el conjunto de datos por su nombre 
imgs= glob.glob(dataset + "/*.jpg")

: 

## Entornos 

Los códigos de Python se ejecutan en un entorno virtual, que define la versión del intérprete de Python y los paquetes instalados disponibles. Los entornos suelen gestionarse con Conda o Pip. Para mejorar la portabilidad, se suele crear entornos en contenedores **Docker** alojados en objetivos de cómputo (equipos de desarrollo, máquinas virtuales o clusters en la nube).

AzureML gestiona la creación de los entornos y la instalación de paquetes, generalmente mediante contenedores Docker. Puedes especificar los paquetes necesarios y que AzureML cree un entorno para el experimento.

Dentro de las soluciones empresariales, es importante conocer los entornos de ejecución del código. Los entornos están encapsulados en la clase `environment`, que permite crearlos y especificar la configuración de ejecución.

Opciones de gestión de entornos:

- Azure ML: Crea y registra el entorno automáticamente.
- Gestión manual: Crea y registra entornos propios.

##### Creación de un entorno a partir de un archivo de especificación
Podemos utilizar un archivo de especificación Conda o pip para definir los paquetes necesarios en un entorno de Python y utilizarlo para crear un objeto `Environment`.

In [ ]:
# Ejemplo de archivo de especificación de entorno que se podria llamar conda.yml

name: py_env
dependencies:
  - numpy
  - pandas
  - scikit-learn
  - pip:
    - azureml-defaults

In [ ]:
# Codigo para trabajar con el archivo conda.yml cy sus variables de entorno.

from azureml.core import Environment

env = Environment.from_conda_specification(name='training_environment',
                                           file_path='./conda.yml')

##### Creación de un entorno a partir de un entorno de Conda existente

Si ya tenemos un entorno de Conda existente definido en la estación de trabajo, podemos usarlo para definir nuestro nuevo entorno.

In [ ]:
from azureml.core import Environment

env = Environment.from_existing_conda_environment(name='training_environment',
                                                  conda_environment_name='py_env')

##### Creación de un entorno mediante la especificación de paquetes
Podemos definir un entorno especificando los paquetes Conda y pip que necesitamos en un objeto CondaDependencies

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment('training_environment')
deps = CondaDependencies.create(conda_packages=['scikit-learn','pandas','numpy'],
                                pip_packages=['azureml-defaults'])
env.python.conda_dependencies = deps

##### Configuración de contenedores de entorno

Normalmente, los entornos para el script del experimento se crean en contenedores. 

En el código siguiente se configura un experimento basado en scripts para hospedar el entorno de entorno creado anteriormente en un contenedor (este es el valor predeterminado a menos que use `DockerConfiguration` con un  atributo `use_docker=False`, en cuyo caso el entorno se crea directamente en el destino de proceso)

In [ ]:
from azureml.core import Experiment, ScriptRunConfig
from azureml.core.runconfig import DockerConfiguration

docker_config = DockerConfiguration(use_docker=True)

script_config = ScriptRunConfig(source_directory='my_folder',
                                script='my_script.py',
                                environment=env,
                                docker_runtime_config=docker_config)

Azure ML usa una biblioteca de imágenes base para contenedores, eligiendo la base adecuada para el destino de proceso que se especifique (por ejemplo, incluida la compatibilidad de Cuda con el proceso basado en GPU). Si creamos imágenes de contenedor personalizadas y las hemos registrado en un registro de contenedor, podemos invalidar las imágenes base creadas y usar las nuestras propias modificando los atributos de la propiedad docker del entorno.

In [ ]:
env.docker.base_image='my-base-image'
env.docker.base_image_registry='myregistry.azurecr.io/myimage'

: 

Como alternativa, podemos crear una imagen a petición en función de la imagen base y la configuración adicional de un `dockerfile`

In [ ]:
env.docker.base_image = None
env.docker.base_dockerfile = './Dockerfile'

Por defecto, Azure Machine Learning controla las rutas de acceso de Python y las dependencias de paquetes.
Si la imagen ya incluye una instalación de Python con las dependencias que necesita, podemos invalidar este comportamiento estableciendo `python.user_managed_dependencies=True` y estableciendo una ruta de acceso explícita de Python para la instalación.

In [ ]:
env.python.user_managed_dependencies=True
env.python.interpreter_path = '/opt/miniconda/bin/python'

#### Registro y reutilización de entornos
Con el entorno creado, podemos registrarlo en el área de trabajo y reutilizarlo para futuros experimentos que tengan las mismas dependencias de Python.

##### Registro de un entorno
usamos el metodo `register` del objeto `Environment` para registrar un entorno

In [ ]:
env.register(workspace=ws)

Podemos ver los entornos registrados de la siguiente manera

In [ ]:
from azureml.core import Environment

env_names = Environment.list(workspace=ws)
for env_name in env_names:
    print('Name:',env_name)

##### Recuperación y uso de un entorno
Podemos recuperar un entorno registrado mediante el método `get` de la  clase `Environment` y, a continuación, asignarlo a `ScriptRunConfig`.

In [ ]:
#  En este ejemplo se recupera el entorno registrado training_environment y se asigna a una configuración de ejecución de script:

from azureml.core import Environment, ScriptRunConfig

training_env = Environment.get(workspace=ws, name='training_environment')

script_config = ScriptRunConfig(source_directory='my_folder',
                                script='my_script.py',
                                environment=training_env)

## Objetivos de cómputo en Azure ML

Son computadoras físicas o virtuales donde se ejecutan experimentos. Existen distintos tipos para adaptarse a tus necesidades.

1. Tipos de objetivos de cómputo
    - **Cómputo local**: Ideal para desarrollo y pruebas con poca data.
        Se ejecuta en el mismo dispositivo donde inicias el experimento (ej. nuestro pc, en la estacion de trabajo del notebook...).
    - **Clúster de cómputo**: Para alta escalabilidad con mucha data o procesamiento paralelo.
        Grupos de máquinas virtuales que se expanden o contraen según la demanda/necesidad.
    - **Cómputo adjunto**: Aprovecha entornos de cómputo ya existentes en Azure (ej. máquinas virtuales, Databricks).
        Útil para cargas de trabajo específicas.

2. Objetivos de cómputo para inferencia (solo para Azure ML Studio)
    - **Clústeres de inferencia**: Utilizan Azure Kubernetes Service para desplegar modelos entrenados como servicios de inferencia.

3. Beneficios de usar objetivos de cómputo
    - **Flexibilidad**: Desarrollar y probar en local, luego escalar a producción.
    - **Optimización de recursos**: Ejecutar procesos en el objetivo más adecuado (ej. CPU para entrenar, CPU solo para evaluar).
    - **Control de costes**: Pagar solo por el uso, iniciar y detener objetivos automáticamente, escalado automático.


#### 1. Creación de objetivos de computo

Las maneras más comunes de crear o asociar un compute target son, usar la página Compute en Azure ML Studio o usar el SDK.

##### - Creación de un objetivo de computo administrado con el SDK 
    
Es aquel por Azure ML, como un clúster de proceso de Azure Machine Learning.

Para crear un clúster de proceso de Azure Machine Learning, use la  clase `azureml.core.compute.ComputeTarget` y la  clase `AmlCompute`, como se muestra a continuación.

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Specify a name for the compute (unique within the workspace)
compute_name = 'aml-cluster'

# Define compute configuration
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',      # Creamos un objeto AmlCompute con la configuración necesaria   
                                                       min_nodes=0, max_nodes=4,        # Definimos el número mínimo y máximo de nodos
                                                       vm_priority='dedicated')         # Definimos la prioridad del clúster

# Create the compute
aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)                    # Creamos el clúster de cálculo
aml_cluster.wait_for_completion(show_output=True)                                       # Esperamos a que se complete la creación

##### - Adjuntar un objetivo de computo no gestionado con el SDK

Un compute target no gestionado es aquel que se define y gestiona fuera del espacio de trabajo de Azure ML; por ejemplo, una máquina virtual Azure o un clúster Azure Databricks.

El codigo usado para gestionar los compute targets no administrados es similar al de cómputo administrado, excepto que debe utilizar el método `ComputeTarget.attach()` para adjuntar el cómputo existente en función de sus ajustes de configuración específicos del objetivo.

Por ejemplo, este codigo se puede utilizar para adjuntar un clúster Azure Databricks existente

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, DatabricksCompute

ws = Workspace.from_config()

compute_name = 'db_cluster'

db_workspace_name = 'db_workspace'
db_resource_group = 'db_resource_group'
db_access_token = '1234-abc-5678-defg-90...'
db_config = DatabricksCompute.attach_configuration(resource_group=db_resource_group,        # Creamos un objeto DatabricksCompute con la configuración necesaria
                                                   workspace_name=db_workspace_name,        # Definimos el nombre del clúster de Databricks
                                                   access_token=db_access_token)            # Definimos el token de acceso

# Create the compute
databricks_compute = ComputeTarget.attach(ws, compute_name, db_config)                      # Creamos el clúster de Databricks
databricks_compute.wait_for_completion(True)

##### - Comprobación de la existencia de objetivos de cálculo

Si se quiere comprobar la existencia de un objetivo de cálculo y crear uno nuevo sólo si no hay ninguno con el nombre especificado. 
Para esto, podemos capturar la excepción `ComputeTargetException`

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_name = "aml-cluster"

# Check if the compute target exists
try:
    aml_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                           max_nodes=4)
    aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)

aml_cluster.wait_for_completion(show_output=True)

#### 2. Uso de los objetivos de computo

Una vez se hayan creado o asociado los compute targets en la workstation, puede usarlos para ejecutar cargas de trabajo específicas; como son los experimentos.

Para usar un objetivo de computo determinado, puede especificarlo en el parámetro adecuado para una configuración de ejecución de experimentos. 

Por ejemplo, el siguiente codigo configura un estimador para usar el destino de proceso denominado aml-cluster

In [ ]:
from azureml.core import Environment, ScriptRunConfig

compute_name = 'aml-cluster'

training_env = Environment.get(workspace=ws, name='training_environment')

script_config = ScriptRunConfig(source_directory='my_dir',                      # Creamos un objeto ScriptRunConfig con la configuración necesaria
                                script='script.py',                             # Definimos el directorio y el script
                                environment=training_env,                       # Definimos el entorno
                                compute_target=compute_name)                    # Definimos el clúster de cálculo

Cuando se envía un experimento, la ejecución se pondrá en cola mientras se inicia el objetivo de computación aml-cluster y se crea en él el entorno especificado, y luego la ejecución se procesará en el entorno de computación.

En lugar de especificar el nombre del objetivo de computación, puede especificar un objeto `ComputeTarget`

In [ ]:
from azureml.core import Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget

compute_name = "aml-cluster"

training_cluster = ComputeTarget(workspace=ws, name=compute_name)

training_env = Environment.get(workspace=ws, name='training_environment')

script_config = ScriptRunConfig(source_directory='my_dir',                      # Creamos un objeto ScriptRunConfig con la configuración necesaria
                                script='script.py',                             # Definimos el directorio y el script
                                environment=training_env,                       # Definimos el entorno
                                compute_target=training_cluster)                # Definimos el clúster de cálculo con compute_target

## Pipelines

En Azure ML, las tareas se ejecutan como experimentos utilizando recursos informáticos y datos.

Para los procesos empresariales de ciencia de datos, se recomienda dividir el proceso en tareas individuales y orquestarlas con Pipelines (secuencias de pasos conectados).

**Los Pipelines son clave para implementar una solución MLOps efectiva.**

- Aclaración sobre el término "Pipeline"
    
    El término "Pipeline" se usa mucho en ML con significados diferentes.

    - **Scikit-learn**: enlaza preprocesamiento de datos con algoritmos de entrenamiento.
    - **Azure DevOps**: automatiza tareas de compilación y configuración de software.
    
    Es posible tener un Pipeline de Azure DevOps que ejecute un Pipeline de Azure ML, el cual puede incluir pasos para que entrene un modelo basado en un Pipeline de Scikit-learn.

1. Concepto de Pipeline
    - Es un flujo de trabajo compuesto de tareas de aprendizaje automático.
    - Cada tarea se implementa como un paso (secuencial o paralelo).
    - Permite construir lógica de flujo sofisticada para organizar operaciones de aprendizaje automático.
    
2. Ejecución de pasos
    - Cada paso se ejecuta en un objetivo de cómputo específico.
    - Se pueden combinar diferentes tipos de procesamiento para lograr un objetivo general.

3. Ejecución del Pipeline
    - Se ejecuta como un experimento.
    - Cada paso se ejecuta en su objetivo asignado como parte del experimento.

4. Componentes de un Pipeline
    - Consta de uno o más pasos que realizan tareas.
    - Azure ML admite muchos tipos de pasos:
      - `PythonScriptStep`: Ejecuta un script de Python específico.
      - `DataTransferStep`: Copia datos entre almacenes mediante Azure Data Factory.
      - `DataBrickStep`: Ejecuta un script de notebook o un JAR compilado en un cluster de Databricks.
      - `AdlaStep`: Ejecuta un trabajo de SQL en Azure Data Lake Analytics.
      - `ParallelRunStep`: Ejecuta un script de Python como una tarea distribuida en múltiples nodos de cómputo.
      - Ver la documentación para una lista completa de tipos de pasos compatibles.

5. Creación de un Pipeline
   - Se requiere definir cada paso primero.
   - Luego se crea un pipeline que incluye los pasos.
   - La configuración específica de cada paso depende del tipo.
     - Ejemplo: definir dos pasos de script de Python para preparar datos y entrenar un modelo.
   - Una vez definidos los pasos, se asignan al pipeline y se ejecutan como un experimento.
  
6. Flujo de trabajo
    - Un pipeline suele tener pasos que dependen de la salida de pasos anteriores.
      - Ejemplo: un script preprocesando datos (paso 1) usados luego para entrenar un modelo (paso 2).

7. Objeto de configuración de conjunto de datos de archivo de salida
    - Objeto especial que referencia una ubicación para almacenamiento intermedio de datos.
    - Crea una dependencia de datos entre pasos del pipeline.
    - Actúa como un almacenamiento intermedio para pasar datos entre pasos.

8. Pasando datos entre los pasos
   - Se usa el objeto de configuración de conjunto de datos de archivo de salida.
   - Debes definir un objeto con nombre que referencie una ubicación en un almacén de datos.
   - Si no se especifica un almacén, se usa el predeterminado.
   - Pasa el objeto como argumento de script en pasos que ejecutan scripts.
   - Incluye código en esos scripts para escribir o leer datos del objeto de argumento.

#### 1. OutputFileDatasetConfig pasos entre Inputs y Outputs

Para utilizar un objeto `OutputFileDatasetConfig` para pasar datos entre los pasos, se debe:

1. Definir un objeto `OutputFileDatasetConfig` con un nombre que haga referencia a una ubicación en un almacén de datos. Si no se especifica un almacén de datos explicitamente, se utilizará el almacén de datos predeterminado.
2. Pasar el objeto `OutputFileDatasetConfig` como argumento en los pasos que ejecutan scripts.
3. Incluya código en esos scripts para escribir en el argumento `OutputFileDatasetConfig` como salida o leerlo como entrada.

Por ejemplo, el siguiente código define un objeto `OutputFileDatasetConfig` que para los datos preprocesados que deben pasarse entre los pasos.

In [ ]:
# Input

from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep

raw_ds = Dataset.get_by_name(ws, 'raw_dataset')

data_store = ws.get_default_datastore()                                                        
prepped_data = OutputFileDatasetConfig('prepped')                                               # Creamos un objeto OutputFileDatasetConfig con la ubicación de salida


step1 = PythonScriptStep(name = 'prepare data',                                                 # Creamos el primer paso del pipeline para ejecutar data_prep.py
                         source_directory = 'scripts',
                         script_name = 'data_prep.py',
                         compute_target = 'aml-cluster',
                         
                         # Los argumentos del script incluyen PipelineData
                         arguments = ['--raw-ds', raw_ds.as_named_input('raw_data'),            # Pasamos el conjunto de datos como argumento
                                      '--out_folder', prepped_data])                            # Pasamos la ubicación de salida como argumento 


step2 = PythonScriptStep(name = 'train model',                                                  # Creamos el segundo paso del pipeline para ejecutar train_model.py                                                             
                         source_directory = 'scripts',
                         script_name = 'train_model.py',
                         compute_target = 'aml-cluster',
                         
                         # Pasamos como argumento del script
                         arguments=['--training-data', prepped_data.as_input()])                # Pasamos la ubicación de salida como argumento

En los propios scripts, se puede obtener una referencia al objeto `OutputFileDatasetConfig` desde el argumento, y utilizarlo como una carpeta local.

In [ ]:
# Output

from azureml.core import Run
import argparse
import os

run = Run.get_context()

parser = argparse.ArgumentParser()                                                              # Creamos un objeto ArgumentParser para manejar los argumentos del script
parser.add_argument('--raw-ds', type=str, dest='raw_dataset_id')
parser.add_argument('--out_folder', type=str, dest='folder')
args = parser.parse_args()
output_folder = args.folder

raw_df = run.input_datasets['raw_data'].to_pandas_dataframe()                                   # Obtenemos el conjunto de datos por su nombre y lo convertimos a un DataFrame de pandas

# code to prep data (in this case, just select specific columns)
prepped_df = raw_df[['col1', 'col2', 'col3']]

# Save prepped data to the PipelineData location
os.makedirs(output_folder, exist_ok=True)
output_path = os.path.join(output_folder, 'prepped_data.csv')
prepped_df.to_csv(output_path)

#### 2. Reutilizar pasos de pipeline

Las canalizaciones con varios pasos de larga duración pueden tardar mucho tiempo en completarse. Azure ML incluye algunas características de almacenamiento en caché y reutilización para reducir estos tiempos.

- **Gestión de la reutilización de la salida de pasos**
  
    De forma predeterminada, la salida del paso de una ejecución de pipeline anterior se reutiliza sin volver a ejecutar el paso, siempre que el script, el directorio de origen y otros parámetros del paso no hayan cambiado. La reutilización de pasos puede reducir el tiempo que se tarda en ejecutar cada pipeline, pero tambien puede dar lugar a resultados obsoletos cuando no se han tenido en cuenta los cambios en los datos posteriores.

    Para controlar la reutilización de un paso individual, puede establecer el  parámetro `allow_reuse` en la configuración del paso, de la siguiente manera:

In [ ]:
step1 = PythonScriptStep(name = 'prepare data',
                         source_directory = 'scripts',
                         script_name = 'data_prep.py',
                         compute_target = 'aml-cluster',
                         runconfig = run_config,
                         inputs=[raw_ds.as_named_input('raw_data')],
                         outputs=[prepped_data],
                         arguments = ['--folder', prepped_data]),
                         # Disable step reuse
                         allow_reuse = False)

- **Forzar la ejecución de todos los pasos**
    Cuando tiene varios pasos, podemos forzar la ejecución de todos ellos independientemente de la configuración de reutilización individual, estableciendo el parámetro `regenerate_outputs` al enviar el experimento de pipeline:

In [ ]:
pipeline_run = experiment.submit(train_pipeline, regenerate_outputs=True)

#### 3. Publicar pipelines

Después de crear una canalización, puede publicarla para crear un punto de conexión REST a través del cual se pueda ejecutar la canalización a petición.

- **Publicación de una canalización**

    Para publicar una canalización, puede llamar a su  método de publicación:

In [ ]:
published_pipeline = pipeline.publish(name='training_pipeline',
                                          description='Model training pipeline',
                                          version='1.0')

Como alternativa, puede llamar al método de publicación en una ejecución correcta de la canalización:

In [ ]:
# Get the most recent run of the pipeline
pipeline_experiment = ws.experiments.get('training-pipeline')
run = list(pipeline_experiment.get_runs())[0]

# Publish the pipeline from the run
published_pipeline = run.publish_pipeline(name='training_pipeline',
                                          description='Model training pipeline',
                                          version='1.0')

Una vez publicada la canalización, puede verla en Azure Machine Learning Studio. También puede determinar el URI de su punto de conexión de la siguiente manera:

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

- **Uso de una canalización publicada**

    Para iniciar un punto de conexión publicado, realice una solicitud HTTP a su punto de conexión REST, pasando un encabezado de autorización con un token para una entidad de servicio con permiso para ejecutar la canalización y una carga JSON que especifique el nombre del experimento. La canalización se ejecuta de forma asincrónica, por lo que la respuesta de una llamada REST correcta incluye el identificador de ejecución. Puede usarlo para realizar un seguimiento de la ejecución en Azure Machine Learning Studio.

Por ejemplo, el siguiente código de Python realiza una solicitud REST para ejecutar una canalización y muestra el identificador de ejecución devuelto.

In [ ]:
import requests

response = requests.post(rest_endpoint,
                         headers=auth_header,
                         json={"ExperimentName": "run_training_pipeline"})
run_id = response.json()["Id"]
print(run_id)

#### 4. Uso de parámetros de pipeline

Podemos aumentar la flexibilidad de una canalización definiendo parámetros.

- **Definición de parámetros para una canalización**

    Para definir parámetros para una canalización, cree un  objeto PipelineParameter para cada parámetro y especifique cada parámetro en al menos un paso.

    NOTA: Debemos definir los parámetros de una canalización antes de publicarla!

Por ejemplo, puede usar el siguiente código para incluir un parámetro para una tasa de regularización en el script utilizado por un estimador:

In [ ]:
from azureml.pipeline.core.graph import PipelineParameter

reg_param = PipelineParameter(name='reg_rate', default_value=0.01)

...

step2 = PythonScriptStep(name = 'train model',
                         source_directory = 'scripts',
                         script_name = 'data_prep.py',
                         compute_target = 'aml-cluster',
                         # Pass parameter as script argument
                         arguments=['--in_folder', prepped_data,
                                    '--reg', reg_param],
                         inputs=[prepped_data])

- **Ejecución de una canalización con un parámetro**

Después de publicar una canalización con parámetros, puede pasar valores de parámetro en la carga JSON para la interfaz REST:

In [ ]:
response = requests.post(rest_endpoint,
                         headers=auth_header,
                         json={"ExperimentName": "run_training_pipeline",
                               "ParameterAssignments": {"reg_rate": 0.1}})

#### 5. Programar Pipelines

Después de publicar una canalización, puede iniciarla a petición a través de su punto de conexión REST, o puede hacer que la canalización se ejecute automáticamente en función de una programación periódica o en respuesta a actualizaciones de datos.

- **Programación de una canalización para intervalos periódicos**

    Para programar una canalización para que se ejecute a intervalos periódicos, debe definir un ScheduleRecurrence que determine la frecuencia de ejecución y usarlo para crear un Schedule.

Por ejemplo, el código siguiente programa una ejecución diaria de una canalización publicada.

In [ ]:
from azureml.pipeline.core import ScheduleRecurrence, Schedule

daily = ScheduleRecurrence(frequency='Day', interval=1)
pipeline_schedule = Schedule.create(ws, name='Daily Training',
                                        description='trains model every day',
                                        pipeline_id=published_pipeline.id,
                                        experiment_name='Training_Pipeline',
                                        recurrence=daily)

- **Desencadenar una ejecución de canalización en los cambios de datos**

    Para programar una canalización para que se ejecute cada vez que cambien los datos, debe crear una programación que supervise una ruta de acceso especificada en un almacén de datos, como se muestra a continuación:

In [ ]:
from azureml.core import Datastore
from azureml.pipeline.core import Schedule

training_datastore = Datastore(workspace=ws, name='blob_data')
pipeline_schedule = Schedule.create(ws, name='Reactive Training',
                                    description='trains model on data change',
                                    pipeline_id=published_pipeline_id,
                                    experiment_name='Training_Pipeline',
                                    datastore=training_datastore,
                                    path_on_datastore='data/training')

## Implementacion de servicios de aprendizaje automatico en tiempo real con Azure ML

- **Inferencia**: Uso de un modelo entrenado para predecir etiquetas para datos nuevos (no usados durante el entrenamiento).

- **Servicio de Inferencia en Tiempo Real**: Permite a las aplicaciones solicitar predicciones inmediatas al modelo para datos individuales o pequeños conjuntos.

Despliegue del Modelo:

- **Contenedorizado en AKS (Azure Kubernetes Services)**: Plataforma de orquestación de contenedores para implementar y administrar aplicaciones en contenedores.

- **Servicio de Inferencia**: Punto de acceso para que las aplicaciones consuman el modelo.

Un modelo puede implementarse como un servicio web en tiempo real en varios destinos, incluyendo localmente, Azure ML, ACI, AKS, una función de Azure o un módulo IoT. Azure ML contenedores para la implementación, empaquetando el modelo y el código en una imagen que se puede implementar en un contenedor en el destino seleccionado.

Nota

    La implementación en un servicio local, una instancia informática o una ACI es una buena opción para las pruebas y el desarrollo. Para producción, se debe implementar en un destino que satisfaga las necesidades específicas de rendimiento, escalabilidad y seguridad de la arquitectura de la aplicación.

Para implementar un modelo como un servicio de inferencia en tiempo real, debe realizar las siguientes tareas:

#### 1. Registro de un modelo entrenado
Después de entrenar correctamente un modelo, debe registrarlo en el área de trabajo de Azure ML. Su servicio en tiempo real podrá cargar el modelo cuando sea necesario.

Para registrar un modelo a partir de un archivo local, puede utilizar el método `register` del  objeto Model como se muestra:

In [ ]:
# Cuando tienes un modelo que se ha entrenado fuera de Azure ML y quieres registrar el modelo en tu espacio de trabajo.

from azureml.core import Model

classification_model = Model.register(workspace=ws,                         # Registramos el modelo en el espacio de trabajo
                       model_name='classification_model',
                       model_path='model.pkl',                              # ruta local
                       description='A classification model')

Alternativamente, si tenemos referencia al `run` utilizado para entrenar el modelo, podemos utilizar su método `register_model`

In [ ]:
# run en Azure ML se refiere a una ejecución individual de un script de entrenamiento o un pipeline.
# Cuando se llama a run.register_model, el modelo se registra junto con metadatos sobre el Run específico, como los parámetros de entrenamiento utilizados. 
# Esto puede ser útil para rastrear cómo se creó el modelo.

run.register_model( model_name='classification_model',                      # Registramos el modelo en el espacio de trabajo
                    model_path='outputs/model.pkl',                         # run ruta de salida
                    description='A classification model')

#### 2. Definir una configuración de inferencia

Los modelos que se implementan como un servicio, constan de:

1. Un script para cargar el modelo y devolver predicciones para los datos enviados.
2. Un entorno en el que se ejecutará el script.

##### Crear un script de entrada

Creando la secuencia de comandos de entrada (tambien llamada **secuencia de comandos de puntuación**), para el servicio como un archivo de Python (.py), necesitamos de:

   - `init()`: Se llama cuando se inicializa el servicio.
   - `run(raw_data)`: Se llama cuando se envían nuevos datos al servicio.

Normalmente, se usa la función `init` para cargar el modelo desde el registro de modelos y se usa la función `run` para generar predicciones a partir de los datos de entrada.

In [ ]:
import json
import joblib
import numpy as np
from azureml.core.model import Model

# llamamos a la función init() para cargar el modelo en la memoria cuando se inicia el contenedor
def init():
    global model
    model_path = Model.get_model_path('classification_model')                   # Obtenemos la ruta del modelo y lo cargamos en memoria
    model = joblib.load(model_path)


# llamamos a la función run() para obtener una predicción para los datos de entrada cuando se realiza una solicitud al servicio web
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])                               # Convertimos los datos de entrada en un array de numpy
    predictions = model.predict(data)                                           # Realizamos las predicciones
    return predictions.tolist()                                                 # Devolvemos las predicciones como una lista serializable en JSON

##### Crear un entorno

Requerimos de un entorno de Python en el que ejecutar el script de entrada, que se puede configurar mediante el archivo de configuración de Conda. 

Una manera sencilla de crear este archivo es usar una clase `CondaDependencies` para crear un entorno predeterminado (que incluye el paquete `azureml-defaults` y paquetes de uso común como `numpy` y `pandas`, pero podemos agregar cualquier otro paquete necesario), a continuación, serealizamos el entorno en una cadena y lo guardamos:

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

# Creamos un objeto CondaDependencies y agregamos las dependencias necesarias
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Guardamos el archivo de especificación de entorno
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

##### Combinar el script y el entorno en un InferenceConfig

Después de crear el script de entrada y el archivo de configuración del entorno, podemos combinarlos en un `InferenceConfig` para el servicio de la siguiente manera

In [ ]:
from azureml.core.model import InferenceConfig

# Creamos un objeto InferenceConfig con la configuración necesaria
classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = 'service_files',
                                              entry_script="score.py",
                                              conda_file="env.yml")

##### Definir una configuración de implementación

Con el script y el archivo de entorno, ahora debemos configurar el proceso en el que se implementará el servicio. 

Si vamos a realizar la implementación en un clúster de AKS, tenemos que crear el clúster y un destino de proceso para él antes de realizar la implementación

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute

# Creamos un objeto AksCompute con la configuración necesaria para el clúster de AKS
cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(location='eastus')
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

Una vez creado el destino del proceso, podemos definir la configuración de implementación. Esta configuración establece la especificación del proceso, específica para el destino, para la implementación en contenedores.

In [ ]:
from azureml.core.webservice import AksWebservice

# Creamos un objeto AksWebservice con la configuración necesaria para el servicio web
classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

Para configurar una implementación de ACI, el código es similar. 

No necesitas crear un destino de proceso de ACI explícitamente y debes usar la clase `deploy_configuration` del espacio de nombres `azureml.core.webservice.AciWebservice`. De manera similar, puedes usar el espacio de nombres `azureml.core.webservice.LocalWebservice` para configurar un servicio local basado en Docker.

Si queremos implementar un modelo en una función de Azure, no necesitas crear una configuración de implementación. En su lugar, debes empaquetar el modelo según el tipo de desencadenador de función que quieras usar. Esta funcionalidad está en versión preliminar en el momento de escribir este artículo. Para obtener más información, consulta la sección [“Implementación de un modelo de aprendizaje automático en Azure Functions”](https://learn.microsoft.com/es-es/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&tabs=azure-cli) en la documentación de Azure ML.

##### Implementar el modelo
Una vez preparada toda la configuración, podemos implementar el modelo. La forma más fácil de hacerlo es llamar al método deploy de la clase `Model`, de la siguiente manera:

In [ ]:
from azureml.core.model import Model

model = ws.models['classification_model']                                       # Obtenemos el modelo por su nombre
service = Model.deploy(workspace=ws,                                            # Desplegamos el servicio web en el clúster de AKS
                       name = 'classifier-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)                                 # Esperamos a que se complete el despliegue

: 

En el caso de ACI o servicios locales, puede omitir el parámetro `deployment_target` (o establecerlo en **None**).

[Más into sobre la implementación de modelos con Azure Machine Learning](https://learn.microsoft.com/es-es/azure/machine-learning/how-to-deploy-online-endpoints?view=azureml-api-2&tabs=azure-cli)

#### 3. Consumo de un servicio de inferencia en tiempo real

Después de implementar el servicio en tiempo real, puede consumirlo desde aplicaciones cliente para predecir etiquetas para nuevos casos de datos.

##### Uso del SDK de Azure Machine Learning

Para realizar pruebas, puede usar el SDK de Azure ML para llamar a un servicio web a través del método run de un objeto `WebService` que haga referencia al servicio implementado. Normalmente, los datos se envían al método `run` en formato JSON con la siguiente estructura:

In [ ]:
{
  "data":[
      [0.1,2.3,4.1,2.0],  // 1st case
      [0.2,1.8,3.9,2.1],  // 2nd case
      ...
  ]
}

La respuesta del método `run` sera una colección JSON con una predicción para cada caso que se envió en los datos.

In [ ]:
# En este ejemplo se llama a un servicio y se muestra la respuesta

import json

# matriz de datos nuevos
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]


json_data = json.dumps({"data": x_new})                     # Convertimos la matriz en un formato JSON serializable

response = service.run(input_data = json_data)              # Llamamos al servicio web con los datos de entrada

predictions = json.loads(response)                          # Convertimos la respuesta en un objeto JSON

for i in range(len(x_new)):                                 # Mostramos las predicciones para cada conjunto de datos
    print (x_new[i], predictions[i])

##### Uso de un punto de conexión REST 

En producción, la mayoría de las aplicaciones cliente no incluirán el SDK de Azure ML y consumirán el servicio a través de su interfaz REST. 

Podemos determinar el punto de conexión de un servicio implementado en Azure ML Studio o recuperar la propiedad `scoring_uri` del objeto Webservice en el SDK

In [ ]:
# En este ejemplo se muestra cómo obtener la dirección URL del servicio web

endpoint = service.scoring_uri
print(endpoint)

Con el punto de conexión conocido, puede usar una solicitud HTTP POST con datos JSON para llamar al servicio. En el siguiente ejemplo se muestra cómo hacerlo con Python:

In [ ]:
import requests
import json

x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

json_data = json.dumps({"data": x_new})                         # Convertimos la matriz en un formato JSON serializable

request_headers = { 'Content-Type':'application/json' }         # Set the content type in the request headers

response = requests.post(url = endpoint,                        # Realizamos la solicitud POST al servicio web
                         data = json_data,
                         headers = request_headers)

predictions = json.loads(response.json())                       # Convertimos la respuesta en un objeto JSON

for i in range(len(x_new)):                                     # Imprimimos las predicciones para cada conjunto de datos
    print ((x_new[i]), predictions[i])

##### Autenticación

En producción, es probable que queramos restringir el acceso a los servicios mediante la aplicación de autenticación. Hay dos tipos de autenticación que se pueden aplicar:
- **Clave**: las solicitudes se autentican especificando la clave asociada al servicio.
- **Token**: se autentican proporcionando un token web JSON (JWT).

De forma predeterminada, la autenticación está deshabilitada para los servicios ACI y se establece en la autenticación basada en claves para los servicios de AKS (para los que las claves principal y secundaria se generan automáticamente). Opcionalmente, puede configurar un servicio de AKS para usar la autenticación basada en tokens (que no es compatible con los servicios ACI).

Suponiendo que tenemos una sesión autenticada establecida con el área de trabajo, podemos recuperar las claves del servicio mediante el método `get_keys` del objeto WebService asociado al servicio:

In [ ]:
primary_key, secondary_key = service.get_keys()

En el caso de la autenticación basada en tokens, la aplicación cliente debe usar la autenticación de entidad de servicio para comprobar su identidad a través de Azure Active Directory (Azure AD) y llamar al  método `get_token` del servicio para recuperar un token de tiempo limitado.

Para realizar una llamada autenticada al punto de conexión REST del servicio, debemos incluir la clave o el token en el encabezado de la solicitud de la siguiente manera

In [ ]:
import requests
import json

x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

json_data = json.dumps({"data": x_new})

request_headers = { "Content-Type":"application/json",                      # incluimos la clave de autenticación en la solicitud
                    "Authorization":"Bearer " + key_or_token }

response = requests.post(url = endpoint,                                    # llama al servicio web con los datos de entrada
                         data = json_data,
                         headers = request_headers)

predictions = json.loads(response.json())

for i in range(len(x_new)):
    print ((x_new[i]), predictions[i])

#### 5. Implementación de un modelo como servicio web en tiempo real

Destinos de proceso
- Proceso local
- Instancia de proceso de Azure Machine Learning
- Instancia de Azure Container Instance (ACI)
- Clúster de Azure Kubernetes Service (AKS)
- Función de Azure
- Módulo de Internet de las cosas (IoT)

Mecanismo de implementación
- Contenedores
- Empaquetado del modelo y el código como una imagen
- Implementación en un contenedor en el destino elegido

Consideraciones:
- Pruebas y desarrollo: servicio local, instancia informática o ACI
- Producción: destino que cumpla con los requisitos de rendimiento, escalabilidad y seguridad de la aplicación

Tareas para implementar un servicio de inferencia en tiempo real:
1. **Registrar el modelo y el entorno**: Almacenar el modelo y su entorno de ejecución en Azure Machine Learning.
2. **Crear una configuración de inferencia**: Especificar cómo se ejecuta el modelo en tiempo real.
3. **Crear una configuración de destino**: Seleccionar el destino de proceso y las opciones de implementación.
4. **Implementar el servicio**: Implementar el modelo como un servicio web en el destino elegido.
5. **Probar el servicio**: Enviar solicitudes de prueba al servicio y verificar las predicciones.